In [1]:
PATH = '/home/g056122/FeatExt_Data_Clustering/Final/collaborative_filtering'

In [2]:
DATAPATH = '/home/g056122/ISG_studienarbeit/Movielens_100K'

In [3]:
# import required libraries
import pandas as pd
from kmodes.kprototypes import KPrototypes
import matplotlib.pyplot as plt

In [4]:
# import user data
dfusers = pd.read_csv(f"{DATAPATH}/u.user", sep='|', header=None, engine='python', encoding='latin-1')
dfusers.columns = ['userId', 'age', 'gender', 'occupation', 'zipcode']

In [5]:
dfusers.head()

,userId,age,gender,occupation,zipcode
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [6]:
# encode users dataset
categorylist=['gender', 'occupation', 'zipcode']
n_dfusers = pd.get_dummies(dfusers, columns=categorylist)

In [7]:
n_dfusers.head()

,userId,age,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,...,zipcode_N2L5N,zipcode_N4T1A,zipcode_R3T5K,zipcode_T8H1N,zipcode_V0R2H,zipcode_V0R2M,zipcode_V1G4L,zipcode_V3N4P,zipcode_V5A2B,zipcode_Y1A6B
0,1,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,53,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,23,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,24,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,33,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
n_dfusers.describe()

,userId,age,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,...,zipcode_N2L5N,zipcode_N4T1A,zipcode_R3T5K,zipcode_T8H1N,zipcode_V0R2H,zipcode_V0R2M,zipcode_V1G4L,zipcode_V3N4P,zipcode_V5A2B,zipcode_Y1A6B
count,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,...,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000
mean,472.000000,34.051962,0.289502,0.710498,0.083775,0.029692,0.007423,0.100742,0.071050,0.019088,...,0.001060,0.001060,0.001060,0.002121,0.001060,0.001060,0.001060,0.001060,0.001060,0.001060
std,272.364951,12.192740,0.453772,0.453772,0.277197,0.169828,0.085883,0.301147,0.257044,0.136907,...,0.032564,0.032564,0.032564,0.046029,0.032564,0.032564,0.032564,0.032564,0.032564,0.032564
min,1.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,236.500000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,472.000000,31.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,707.500000,43.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,943.000000,73.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
category_index = list(range(2,n_dfusers.shape[1]+1))

In [10]:
# Elbow curve to find optimal K
cost = []
K = range(1, 10)
for num_clusters in list(K):
    kprototype = KPrototypes(n_clusters=num_clusters, init = "Huang")
    kprototype.fit_predict(n_dfusers, categorical = category_index)
    cost.append(kprototype.cost_)

AssertionError: Categorical index larger than number of columns.